In [ ]:
from modules.lstm_encoder_decoder import LSTMEncoderDecoder
from keras.regularizers import l2
from keras.layers import LSTM, SimpleRNN, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation, BatchNormalization
import os
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex, HTML, clear_output
import ipy_table
import ipywidgets as widgets
from scipy import ndimage
from matplotlib import pyplot as plt

## Classes

In [ ]:
def get_classnames(classes, classnames):
    class_to_classname = {}
    for cl, classname in zip(true_classes, true_classnames):
        class_to_classname[cl] = classname

    unique_classnames = []
    for i in range(len(class_to_classname)):
        unique_classnames.append(class_to_classname[i])
        
    return np.array(unique_classnames)

In [ ]:
indices = np.load('../input/features/lstm/3_steps/indices_dev.npy')
true_classes = np.load('../input/features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('../input/features/res_net/classnames_dev_make-model.npy')[indices]
true_filenames = np.load('../input/features/res_net/filenames_dev.npy')[indices]
unique_classnames = get_classnames(true_classes, true_classnames)

## Predictions

In [ ]:
def map_predictions_to_classnames(predictions, true_classes, classnames):
    classes, indices = np.unique(true_classes, return_index=True)
    
    class_to_index = {}
    for cl, index in zip(classes, indices):
        class_to_index[cl] = index
    
    predicted_classnames = list(map(lambda cl: classnames[class_to_index[cl]], predictions))
    return np.array(predicted_classnames)

def map_indices_to_classnames(indices, classnames):
    return np.array(list(map(lambda i: classnames[i], indices)))

#### LSTM Encoder-Decoder

In [ ]:
encoder_decoder = LSTMEncoderDecoder(
    source_path='../input/features/lstm/3_steps/',
    weights_path='../input/convnet_weights/weights.04-0.36-87.85.hdf5',
)

In [ ]:
predictions, probs = encoder_decoder.predict('dev')

In [ ]:
predicted_classnames = map_predictions_to_classnames(predictions, true_classes, true_classnames)

In [ ]:
predicted_probs = np.max(probs, axis=1)
true_class_probs = probs[list(range(len(true_classes))), true_classes]
five_top_classes = np.flip(np.argsort(probs, axis=1), axis=1)[:,0:5]
five_top_probs = np.array(list(map(lambda f: str(f), np.flip(np.sort(probs, axis=1), axis=1)[:,0:5])))
five_top_names = np.array(list(map(lambda f: str(f), map_indices_to_classnames(five_top_classes, unique_classnames))))
five_top_correct = list(map(lambda v: np.isin(v[0], v[1]), zip(true_classes, five_top_classes)))

## Reports creation

In [ ]:
classes, indices = np.unique(true_classes, return_index=True)
classnames = true_classnames[indices]
directories = list(map(lambda f: f.split('/')[0],true_filenames[indices]))

In [ ]:
classes_report = pd.DataFrame({
    'name': classnames,
    'directory': directories,
}, index = classes)

In [ ]:
pred_classnames = list(map(lambda cl: classes_report['name'][cl], predictions))

In [ ]:
samples_report = pd.DataFrame({
    'filename': true_filenames,
    'prediction': predictions,
    'real_class': true_classes,
    'predicted_name': pred_classnames,
    'real_name': true_classnames,
    'predicted_correctly': predictions == true_classes,
    'five_top_prediction_correct': five_top_correct,
    'predicted_prob': predicted_probs,
    'true_class_prob': true_class_probs,
    'five_top_names': five_top_names,
    'five_top_probs': five_top_probs,
})

In [ ]:
classes_count = samples_report['real_class'].value_counts()
classes_report['count'] = classes_count.sort_index()

In [ ]:
correct_count = samples_report[samples_report['predicted_correctly'] == True]['real_class'].value_counts()
classes_report['correct_predictions_count'] = correct_count.sort_index()

In [ ]:
classes_report['correct_predictions_percent'] = classes_report['correct_predictions_count'] / classes_report['count']

In [ ]:
print ('Accuracy', samples_report[samples_report['predicted_correctly'] == True].shape[0] / samples_report.shape[0])
print ('Five top accuracy', samples_report[samples_report['five_top_prediction_correct'] == True].shape[0] / samples_report.shape[0])

## Classes report

In [ ]:
def clear_report():
    clear_output()
    display(widgets.HBox((dropdown_sort, dropdown_direction)))

def display_classes_report(sortBy, direction):
    html = pd.DataFrame.to_html(classes_report.sort_values(sortBy, ascending=direction))
    display(HTML(html))
    
def clear_and_display_report(sortBy, direction):
    clear_report()
    display_classes_report(sortBy, direction)

In [ ]:
dropdown_sort = widgets.Dropdown(
    options=['correct_predictions_percent', 'name', 'count'],
    value='correct_predictions_percent',
    description='Sort by:',
    disabled=False,
)

dropdown_direction = widgets.Dropdown(
    options=['ascending', 'descending'],
    value='ascending',
    description='Direction:',
    disabled=False,
)

dropdown_sort.observe(lambda v: 
                          clear_and_display_report(v['new'], dropdown_direction.value == 'ascending') 
                          if v['name'] == 'value'
                          else None
                     )
dropdown_direction.observe(lambda v: 
                              clear_and_display_report(dropdown_sort.value, v['new'] == 'ascending') 
                              if v['name'] == 'value'
                              else None
                          )

clear_and_display_report(dropdown_sort.value, dropdown_direction.value == 'ascending')

## Samples report

In [ ]:
dropdown_classes = None
dropdown_correctness = None
dropdown_sample = None

def show_sample(row):
    clear_output()
    display(widgets.HBox((dropdown_classes, dropdown_correctness)))
    display(dropdown_sample)
    
    image = np.array(plt.imread('../input/car_data/' + row['filename'].iloc[0]))
    plt.imshow(image)
    print('Classified as: ', row['predicted_name'].iloc[0])
    print('Predicted probability: ', row['predicted_prob'].iloc[0])
    print('Correct class probability: ', row['true_class_prob'].iloc[0])
    print('Five top predictions: ', row['five_top_names'].iloc[0])
    print('Five top prediction probs: ', row['five_top_probs'].iloc[0])
    filenames = samples_report[samples_report['real_name'] == row['predicted_name'].iloc[0]]['filename']
    
    images = []
    for filename in filenames[0:100]:
        images.append(filename)
        
    fig = plt.figure(figsize=(20,40))
    for k in range(10):
        for i, image in enumerate(images[k*3:k*3+3]):
            img = np.array(plt.imread('../input/car_data/' + image))
            ax = fig.add_subplot(10,3,i + 1 + k * 3)
            ax.imshow(img)


def show_samples(classname, classification):
    global dropdown_sample

    df = samples_report[
        (samples_report['real_name'] == classname) &
        (samples_report['predicted_correctly'] == classification)
    ]

    if df.shape[0] == 0:
        return

    dropdown_sample = widgets.Dropdown(
        options=df['filename'],
        value=df['filename'].iloc[0],
        disabled=False,
    )
    
    dropdown_sample.observe(lambda v: show_sample(df[df['filename'] == v['new']]), names='value')
    
    display(dropdown_sample)
    show_sample(df[df['filename'] == df['filename'].iloc[0]])

def observe_classes(v):
    clear_output()
    display(widgets.HBox((dropdown_classes, dropdown_correctness)))
    show_samples(v['new'], dropdown_correctness.value == 'classified correctly')
    
def observe_correctness(v):
    clear_output()
    show_samples(dropdown_classes.value, v['new'] == 'classified correctly')

dropdown_classes = widgets.Dropdown(
    options=classnames,
    value=classnames[0],
    description='Class:',
    disabled=False,
)

dropdown_correctness = widgets.Dropdown(
    options=['misclassified', 'classified correctly'],
    value='misclassified',
    disabled=False,
)

dropdown_classes.observe(observe_classes, names='value')
dropdown_correctness.observe(observe_correctness, names='value')

display(widgets.HBox((dropdown_classes, dropdown_correctness)))
show_samples(classnames[0], dropdown_correctness.value == 'classified correctly')

## Misclassified as

In [ ]:
def show_misclassifications(classname):
    df = samples_report[(samples_report['predicted_correctly'] == False) & (samples_report['real_name'] == classname)]
    df = pd.DataFrame({'Classified as': df['predicted_name']})
    html = pd.DataFrame.to_html(df)
    display(HTML(html))
    
def on_change(v):
    clear_output()
    display(dropdown)
    show_misclassifications(v['new'])

dropdown = widgets.Dropdown(
    options=classnames,
    value=classnames[0],
    description='Class:',
    disabled=False,
)

dropdown.observe(on_change, names='value')

display(dropdown)
show_misclassifications(classnames[0])